In [1]:
import tensorflow as tf 
import tensorflow_probability as tfp
from tensorflow.keras.datasets import mnist


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns

import argparse
import random
import os
import math


tfd = tfp.distributions

In [3]:
import importlib
import new_custom_layers
import utilities

In [ ]:
# # Hyperparameter setup
# learning_rate = 0.001
# epochs = 100
# batch_size = 128
# num_classes = 10
# num_samples = 1

In [20]:

noise = 0.2
num_classes = 3
N_per_class = 50
R_per_class = 30
N_val = 50
N_test = 100
seed = 100

N_train = num_classes*N_per_class

x_train, y_train, x_train_ordered, y_train_ordered = create_synthetic_train(num_classes=num_classes, 
                                                                            N_per_class=N_per_class, 
                                                                            input_noise=noise, R_per_class=R_per_class,
                                                                                     seed=seed)

x_val, y_val = create_synthetic_test(num_classes=num_classes, N_per_class=N_val, input_noise=noise,seed=seed)
x_test, y_test = create_synthetic_test(num_classes=num_classes, N_per_class=N_test, input_noise=noise,seed=seed)

y_train = np.argmax(y_train, 1)
y_test = np.argmax(y_test, 1)
y_val = np.argmax(y_val, 1)

In [11]:
seed = 0 #0
np.random.seed(seed=seed)
random.seed(seed)
batch_size = 50
epochs = 200

In [9]:
M = 2.0000
loss_mat = np.array([[0,1,2],[0.8,0,0.7],[0.9,0.6,0]])
print("         TRUTH\n" )
print('P  ',M - loss_mat[0])
print('R  ',M - loss_mat[1])
print('E  ',M - loss_mat[2])

# Set up weight for weighted cross entropy
class_weight = {0 : 1.,
    1: 2,
    2: 2}
util_mat = M - loss_mat

         TRUTH

P   [2. 1. 0.]
R   [1.2 2.  1.3]
E   [1.1 1.4 2. ]


In [28]:
print(util_mat)
train_samples = x_train.shape[0]
num_classes = np.max(y_train)+1

[[2.  1.  0. ]
 [1.2 2.  1.3]
 [1.1 1.4 2. ]]


In [29]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = new_custom_layers.Softmax( 
    num_classes, 
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

In [30]:
model_sm.fit([x_train, y_train], batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/200
150/150 [==============================] - 1s 6ms/sample - loss: 1.2765
Epoch 2/200
150/150 [==============================] - 0s 67us/sample - loss: 1.2806
Epoch 3/200
150/150 [==============================] - 0s 72us/sample - loss: 1.2711
Epoch 4/200
150/150 [==============================] - 0s 58us/sample - loss: 1.2774
Epoch 5/200
150/150 [==============================] - 0s 79us/sample - loss: 1.2717
Epoch 6/200
150/150 [==============================] - 0s 174us/sample - loss: 1.2633
Epoch 7/200
150/150 [==============================] - 0s 65us/sample - loss: 1.2687
Epoch 8/200
150/150 [==============================] - 0s 104us/sample - loss: 1.2692
Epoch 9/200
150/150 [==============================] - 0s 145us/sample - loss: 1.2597
Epoch 10/200
150/150 [==============================] - 0s 54us/sample - loss: 1.2705
Epoch 11/200
150/150 [==============================] - 0s 87us/sample - loss: 1.2601
Epoch 12/200
150/150 [==============================] - 0s 14

150/150 [==============================] - 0s 88us/sample - loss: 1.1514
Epoch 96/200
150/150 [==============================] - 0s 131us/sample - loss: 1.1583
Epoch 97/200
150/150 [==============================] - 0s 165us/sample - loss: 1.1521
Epoch 98/200
150/150 [==============================] - 0s 86us/sample - loss: 1.1527
Epoch 99/200
150/150 [==============================] - 0s 166us/sample - loss: 1.1561
Epoch 100/200
150/150 [==============================] - 0s 189us/sample - loss: 1.1599
Epoch 101/200
150/150 [==============================] - 0s 75us/sample - loss: 1.1586
Epoch 102/200
150/150 [==============================] - 0s 123us/sample - loss: 1.1452
Epoch 103/200
150/150 [==============================] - 0s 67us/sample - loss: 1.1430
Epoch 104/200
150/150 [==============================] - 0s 80us/sample - loss: 1.1458
Epoch 105/200
150/150 [==============================] - 0s 69us/sample - loss: 1.1517
Epoch 106/200
150/150 [==============================] -

150/150 [==============================] - 0s 83us/sample - loss: 1.1169
Epoch 189/200
150/150 [==============================] - 0s 84us/sample - loss: 1.0998
Epoch 190/200
150/150 [==============================] - 0s 68us/sample - loss: 1.1101
Epoch 191/200
150/150 [==============================] - 0s 57us/sample - loss: 1.1100
Epoch 192/200
150/150 [==============================] - 0s 75us/sample - loss: 1.1045
Epoch 193/200
150/150 [==============================] - 0s 58us/sample - loss: 1.0988
Epoch 194/200
150/150 [==============================] - 0s 60us/sample - loss: 1.1114
Epoch 195/200
150/150 [==============================] - 0s 54us/sample - loss: 1.1047
Epoch 196/200
150/150 [==============================] - 0s 71us/sample - loss: 1.1089
Epoch 197/200
150/150 [==============================] - 0s 134us/sample - loss: 1.1099
Epoch 198/200
150/150 [==============================] - 0s 77us/sample - loss: 1.1118
Epoch 199/200
150/150 [==============================] -

In [33]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

Standard:

Accuracy on optimal decision:  0.6633333333333333
Expected loss:  0.6423333333333333


In [35]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = new_custom_layers.DUCSoftmax( 
    num_classes, 
    util_mat,
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

Instructions for updating:
Use `tf.linalg.matmul` instead


In [36]:
model_sm.fit([x_train, y_train], batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/200
150/150 [==============================] - 1s 4ms/sample - loss: 1.0567
Epoch 2/200
150/150 [==============================] - 0s 57us/sample - loss: 1.0918
Epoch 3/200
150/150 [==============================] - 0s 52us/sample - loss: 1.0574
Epoch 4/200
150/150 [==============================] - 0s 75us/sample - loss: 1.0983
Epoch 5/200
150/150 [==============================] - 0s 105us/sample - loss: 1.0826
Epoch 6/200
150/150 [==============================] - 0s 125us/sample - loss: 1.1444
Epoch 7/200
150/150 [==============================] - 0s 170us/sample - loss: 1.1452
Epoch 8/200
150/150 [==============================] - 0s 54us/sample - loss: 1.1025
Epoch 9/200
150/150 [==============================] - 0s 173us/sample - loss: 1.1738
Epoch 10/200
150/150 [==============================] - 0s 48us/sample - loss: 1.0545
Epoch 11/200
150/150 [==============================] - 0s 96us/sample - loss: 1.1236
Epoch 12/200
150/150 [==============================] - 0s 1

150/150 [==============================] - 0s 130us/sample - loss: 0.9979
Epoch 96/200
150/150 [==============================] - 0s 163us/sample - loss: 0.9872
Epoch 97/200
150/150 [==============================] - 0s 61us/sample - loss: 0.9851
Epoch 98/200
150/150 [==============================] - 0s 129us/sample - loss: 0.9935
Epoch 99/200
150/150 [==============================] - 0s 68us/sample - loss: 0.9819
Epoch 100/200
150/150 [==============================] - 0s 122us/sample - loss: 0.9834
Epoch 101/200
150/150 [==============================] - 0s 70us/sample - loss: 0.9861
Epoch 102/200
150/150 [==============================] - 0s 122us/sample - loss: 0.9784
Epoch 103/200
150/150 [==============================] - 0s 74us/sample - loss: 0.9757
Epoch 104/200
150/150 [==============================] - 0s 155us/sample - loss: 0.9788
Epoch 105/200
150/150 [==============================] - 0s 69us/sample - loss: 0.9741
Epoch 106/200
150/150 [==============================] 

150/150 [==============================] - 0s 196us/sample - loss: 0.9340
Epoch 189/200
150/150 [==============================] - 0s 288us/sample - loss: 0.9304
Epoch 190/200
150/150 [==============================] - 0s 144us/sample - loss: 0.9231
Epoch 191/200
150/150 [==============================] - 0s 98us/sample - loss: 0.9348
Epoch 192/200
150/150 [==============================] - 0s 53us/sample - loss: 0.9245
Epoch 193/200
150/150 [==============================] - 0s 103us/sample - loss: 0.9294
Epoch 194/200
150/150 [==============================] - 0s 80us/sample - loss: 0.9201
Epoch 195/200
150/150 [==============================] - 0s 90us/sample - loss: 0.9273
Epoch 196/200
150/150 [==============================] - 0s 113us/sample - loss: 0.9234
Epoch 197/200
150/150 [==============================] - 0s 78us/sample - loss: 0.9291
Epoch 198/200
150/150 [==============================] - 0s 71us/sample - loss: 0.9301
Epoch 199/200
150/150 [=============================

In [37]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

Standard:

Accuracy on optimal decision:  0.6
Expected loss:  0.29333333333333333
